# ID Immobilier - Analyse Exploratoire des Donnees
**Projet Big Data | Marche Immobilier Togolais**

Ce notebook presente l'analyse exploratoire des donnees (EDA) avant la mise en place du pipeline.
Il permet de comprendre la structure, la qualite et les distributions des donnees sources.

---
**Sources analysees :**
- ImmoAsk : 500 annonces
- Facebook Marketplace : 80 annonces
- CoinAfrique : 4 844 annonces
- Valeurs Venales OTR : 354 enregistrements

## 0. Imports et Configuration

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import seaborn as sns
import os
import warnings
warnings.filterwarnings('ignore')

# Configuration graphique
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False
sns.set_palette('Set2')

print('Imports OK')
print(f'pandas : {pd.__version__}')
print(f'numpy  : {np.__version__}')

In [ ]:
# Chemins vers les fichiers sources
BASE_DIR    = os.path.dirname(os.getcwd())
SOURCES_DIR = os.path.join(BASE_DIR, 'data', 'raw', 'sources')

FICHIERS = {
    'ImmoAsk'       : os.path.join(SOURCES_DIR, 'donnees_immobilieres_reformate_ImmoAsk.xlsx'),
    'Facebook'      : os.path.join(SOURCES_DIR, 'Facebook_donnees_immobilieres.xlsx'),
    'CoinAfrique'   : os.path.join(SOURCES_DIR, 'TRD_Cyrille_CoinAfrique_TogoImmobilier.xlsx'),
    'ValeursVenales': os.path.join(SOURCES_DIR, 'valeurs_venales_togo.xlsx'),
}

print('Fichiers configures')
for nom, path in FICHIERS.items():
    existe = 'OK' if os.path.exists(path) else 'MANQUANT'
    print(f'  {nom:20s} : {existe}')

## 1. Chargement et Vue d'ensemble des Donnees

In [ ]:
# Charger toutes les sources
dfs = {}
for nom, path in FICHIERS.items():
    if os.path.exists(path):
        dfs[nom] = pd.read_excel(path)
        dfs[nom]['_source'] = nom
        print(f'{nom:20s} : {len(dfs[nom]):5d} lignes | {len(dfs[nom].columns)} colonnes')
    else:
        print(f'{nom:20s} : FICHIER NON TROUVE')

In [ ]:
# Apercu des colonnes par source
for nom, df in dfs.items():
    print(f'\n=== {nom} ===')
    print(f'Colonnes : {list(df.columns)}')
    print(df.head(3).to_string())

In [ ]:
# Repartition des annonces par source - graphique
sources_annonces = {k: len(v) for k, v in dfs.items() if k != 'ValeursVenales'}

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Barres
colors = ['#2980B9', '#27AE60', '#E67E22']
ax1.bar(sources_annonces.keys(), sources_annonces.values(), color=colors)
ax1.set_title('Nombre d annonces par source', fontsize=14, fontweight='bold')
ax1.set_ylabel('Nombre d annonces')
for i, (k, v) in enumerate(sources_annonces.items()):
    ax1.text(i, v + 50, str(v), ha='center', fontweight='bold')

# Camembert
ax2.pie(sources_annonces.values(), labels=sources_annonces.keys(),
        autopct='%1.1f%%', colors=colors, startangle=90)
ax2.set_title('Repartition en pourcentage', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'repartition_sources.png'), dpi=150, bbox_inches='tight')
plt.show()
print('CoinAfrique represente la grande majorite des donnees (83%)')

## 2. Analyse de la Qualite des Donnees

In [ ]:
# Analyse des valeurs manquantes par source
print('=== VALEURS MANQUANTES PAR SOURCE ===\n')

for nom, df in dfs.items():
    if nom == 'ValeursVenales':
        continue
    print(f'--- {nom} ---')
    missing = df.isnull().sum()
    missing_pct = (missing / len(df) * 100).round(1)
    summary = pd.DataFrame({'Manquants': missing, 'Pourcentage %': missing_pct})
    print(summary[summary['Manquants'] > 0].to_string())
    print()

In [ ]:
# Visualisation des valeurs manquantes
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
sources_plot = ['ImmoAsk', 'Facebook', 'CoinAfrique']

for i, nom in enumerate(sources_plot):
    df = dfs[nom].drop(columns=['_source'])
    missing_pct = (df.isnull().sum() / len(df) * 100)
    missing_pct = missing_pct[missing_pct > 0]
    if len(missing_pct) > 0:
        axes[i].barh(missing_pct.index, missing_pct.values, color='#E74C3C', alpha=0.8)
        axes[i].set_title(f'{nom} - Valeurs manquantes', fontweight='bold')
        axes[i].set_xlabel('% manquant')
    else:
        axes[i].text(0.5, 0.5, 'Aucune valeur\nmanquante', ha='center', va='center',
                     fontsize=14, transform=axes[i].transAxes)
        axes[i].set_title(f'{nom} - Valeurs manquantes', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'valeurs_manquantes.png'), dpi=150, bbox_inches='tight')
plt.show()

## 3. Analyse des Prix

In [ ]:
# Calcul du prix au m2 pour chaque source
for nom in ['ImmoAsk', 'Facebook', 'CoinAfrique']:
    df = dfs[nom].copy()
    prix_col    = 'Prix'
    surface_col = 'Surface'
    
    df[prix_col]    = pd.to_numeric(df[prix_col], errors='coerce')
    df[surface_col] = pd.to_numeric(df[surface_col], errors='coerce')
    df['prix_m2']   = df[prix_col] / df[surface_col]
    
    valides = df[(df['prix_m2'] > 1000) & (df['prix_m2'] < 2_000_000)]
    rejetes = df[~((df['prix_m2'] > 1000) & (df['prix_m2'] < 2_000_000))]
    
    print(f'--- {nom} ---')
    print(f'  Lignes totales          : {len(df)}')
    print(f'  Prix m2 valides (1k-2M) : {len(valides)} ({len(valides)/len(df)*100:.1f}%)')
    print(f'  Prix m2 rejetes         : {len(rejetes)} ({len(rejetes)/len(df)*100:.1f}%)')
    print(f'  Prix m2 moyen (valides) : {valides["prix_m2"].mean():,.0f} FCFA')
    print(f'  Prix m2 median (valides): {valides["prix_m2"].median():,.0f} FCFA')
    print(f'  Prix m2 min             : {valides["prix_m2"].min():,.0f} FCFA')
    print(f'  Prix m2 max             : {valides["prix_m2"].max():,.0f} FCFA')
    print()
    dfs[nom]['prix_m2'] = df['prix_m2']

In [ ]:
# Distribution des prix au m2 par source
fig, axes = plt.subplots(1, 3, figsize=(16, 5))
sources_plot = ['ImmoAsk', 'Facebook', 'CoinAfrique']
colors = ['#2980B9', '#27AE60', '#E67E22']

for i, (nom, color) in enumerate(zip(sources_plot, colors)):
    df = dfs[nom].copy()
    if 'prix_m2' not in df.columns:
        continue
    valides = df[(df['prix_m2'] > 1000) & (df['prix_m2'] < 2_000_000)]['prix_m2']
    
    axes[i].hist(valides, bins=40, color=color, alpha=0.8, edgecolor='white')
    axes[i].axvline(valides.mean(), color='red', linestyle='--', label=f'Moyenne: {valides.mean():,.0f}')
    axes[i].axvline(valides.median(), color='black', linestyle='--', label=f'Mediane: {valides.median():,.0f}')
    axes[i].set_title(f'{nom} - Distribution prix m2', fontweight='bold')
    axes[i].set_xlabel('Prix au m2 (FCFA)')
    axes[i].set_ylabel('Frequence')
    axes[i].legend(fontsize=9)
    axes[i].xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x/1000:.0f}k'))

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'distribution_prix_m2.png'), dpi=150, bbox_inches='tight')
plt.show()
print('Observation : les distributions sont asymetriques (skewed right) - caracteristique des marches immobiliers')

## 4. Analyse Geographique

In [ ]:
# Zones les plus representees dans chaque source
for nom in ['ImmoAsk', 'Facebook', 'CoinAfrique']:
    df = dfs[nom]
    if 'Quartier' in df.columns:
        top_zones = df['Quartier'].value_counts().head(10)
        print(f'\n--- Top 10 zones : {nom} ---')
        print(top_zones.to_string())

In [ ]:
# Graphique des zones communes entre sources
zones_par_source = {}
for nom in ['ImmoAsk', 'Facebook', 'CoinAfrique']:
    df = dfs[nom]
    if 'Quartier' in df.columns:
        zones_par_source[nom] = set(df['Quartier'].str.lower().str.strip().dropna().unique())

# Calcul des intersections
z_immo     = zones_par_source.get('ImmoAsk', set())
z_fb       = zones_par_source.get('Facebook', set())
z_coin     = zones_par_source.get('CoinAfrique', set())
communes   = z_immo & z_fb & z_coin

print(f'Zones uniques ImmoAsk    : {len(z_immo)}')
print(f'Zones uniques Facebook   : {len(z_fb)}')
print(f'Zones uniques CoinAfrique: {len(z_coin)}')
print(f'Zones communes aux 3     : {len(communes)}')
if communes:
    print(f'Exemples                 : {list(communes)[:10]}')

In [ ]:
# Top 15 zones par nombre d annonces (toutes sources combinees)
dfs_annonces = []
for nom in ['ImmoAsk', 'Facebook', 'CoinAfrique']:
    df = dfs[nom].copy()
    if 'Quartier' in df.columns:
        df['zone'] = df['Quartier'].str.lower().str.strip()
        dfs_annonces.append(df[['zone', '_source']])

df_all = pd.concat(dfs_annonces, ignore_index=True)
df_all = df_all[df_all['zone'].notna() & (df_all['zone'] != 'non spécifié')]
df_all = df_all[df_all['zone'].str.len() <= 30]

top_zones_all = df_all['zone'].value_counts().head(15)

fig, ax = plt.subplots(figsize=(12, 7))
top_zones_all.sort_values().plot(kind='barh', ax=ax, color='#2980B9', alpha=0.85)
ax.set_title('Top 15 zones par nombre d annonces (toutes sources)', fontsize=14, fontweight='bold')
ax.set_xlabel('Nombre d annonces')
ax.set_ylabel('Zone')
for i, v in enumerate(top_zones_all.sort_values().values):
    ax.text(v + 5, i, str(v), va='center', fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'top_zones.png'), dpi=150, bbox_inches='tight')
plt.show()

## 5. Analyse par Type de Bien et Type d'Offre

In [ ]:
# Repartition par type de bien
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for i, nom in enumerate(['ImmoAsk', 'Facebook', 'CoinAfrique']):
    df = dfs[nom]
    if 'Type de bien' in df.columns:
        types = df['Type de bien'].value_counts().head(8)
        axes[i].barh(types.index, types.values, color=sns.color_palette('Set2', len(types)))
        axes[i].set_title(f'{nom} - Types de biens', fontweight='bold')
        axes[i].set_xlabel('Nombre d annonces')

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'types_biens.png'), dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
# Repartition Vente vs Location
fig, axes = plt.subplots(1, 3, figsize=(14, 5))

for i, nom in enumerate(['ImmoAsk', 'Facebook', 'CoinAfrique']):
    df = dfs[nom]
    if "Type d'offre" in df.columns:
        offres = df["Type d'offre"].value_counts()
        colors = ['#E67E22' if 'vente' in str(o).lower() else '#2980B9' for o in offres.index]
        axes[i].pie(offres.values, labels=offres.index, autopct='%1.1f%%',
                    colors=colors, startangle=90)
        axes[i].set_title(f'{nom} - Vente vs Location', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(BASE_DIR, 'data', 'vente_vs_location.png'), dpi=150, bbox_inches='tight')
plt.show()

## 6. Analyse des Valeurs Venales Officielles

In [ ]:
# Analyse des valeurs venales OTR
df_vv = dfs.get('ValeursVenales')
if df_vv is not None:
    print('Structure des valeurs venales :')
    print(df_vv.columns.tolist())
    print(f'Lignes : {len(df_vv)}')
    print(df_vv.head(5).to_string())

In [ ]:
if df_vv is not None:
    # Prix au m2 officiel par zone
    col_prix_m2 = 'Valeur/m² (FCFA)'
    col_zone    = 'Quartier'
    
    if col_prix_m2 in df_vv.columns and col_zone in df_vv.columns:
        df_vv[col_prix_m2] = pd.to_numeric(df_vv[col_prix_m2], errors='coerce')
        top_vv = df_vv.groupby(col_zone)[col_prix_m2].mean().sort_values(ascending=False).head(15)
        
        fig, ax = plt.subplots(figsize=(12, 7))
        top_vv.sort_values().plot(kind='barh', ax=ax, color='#27AE60', alpha=0.85)
        ax.set_title('Top 15 zones - Prix officiel au m2 (Valeurs Venales OTR)', fontsize=13, fontweight='bold')
        ax.set_xlabel('Prix au m2 (FCFA)')
        ax.xaxis.set_major_formatter(mticker.FuncFormatter(lambda x, _: f'{x/1000:.0f}k'))
        plt.tight_layout()
        plt.savefig(os.path.join(BASE_DIR, 'data', 'valeurs_venales_top.png'), dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f'Prix officiel moyen : {df_vv[col_prix_m2].mean():,.0f} FCFA/m2')
        print(f'Prix officiel median: {df_vv[col_prix_m2].median():,.0f} FCFA/m2')

## 7. Identification des Donnees Problematiques

In [ ]:
# Identifier les zones invalides - noms trop longs ou descriptifs
print('=== ZONES PROBLEMATIQUES ===\n')

mots_suspects = ['pharmacie', 'cote de', 'juste', 'derriere', 'face', 'avant',
                 'carrefour', 'forever', 'standing', 'clinique', 'goudron']

for nom in ['ImmoAsk', 'Facebook', 'CoinAfrique']:
    df = dfs[nom]
    if 'Quartier' not in df.columns:
        continue
    
    zones = df['Quartier'].astype(str)
    
    # Zones trop longues
    trop_longues = zones[zones.str.len() > 30].unique()
    
    # Zones avec mots suspects
    suspects = zones[zones.str.lower().str.contains('|'.join(mots_suspects), na=False)].unique()
    
    print(f'--- {nom} ---')
    print(f'  Zones > 30 chars  : {len(trop_longues)} - ex: {list(trop_longues[:2])}')
    print(f'  Zones descriptives: {len(suspects)} - ex: {list(suspects[:2])}')
    print()

In [ ]:
# Resumer les decisions de nettoyage
print('=== REGLES DE NETTOYAGE DECIDEES ===\n')
regles = [
    ('Zone > 30 caracteres', 'Rejet -> zone_trop_longue', 'CoinAfrique surtout'),
    ('Zone = non specifie, vide', 'Rejet -> zone_invalide', 'Toutes sources'),
    ('Zone contient mot suspect', 'Rejet -> zone_description_lieu', 'CoinAfrique surtout'),
    ('Prix m2 > 2 000 000 FCFA', 'Rejet -> prix_m2_trop_eleve', 'Prix total confondu avec prix m2'),
    ('Prix m2 < 1 000 FCFA', 'Rejet -> prix_m2_trop_bas', 'Surface en hectares ou erreur'),
]
for regle, action, raison in regles:
    print(f'  Regle  : {regle}')
    print(f'  Action : {action}')
    print(f'  Raison : {raison}')
    print()

## 8. Synthese et Conclusions

In [ ]:
# Tableau de synthese final
print('=' * 60)
print('SYNTHESE DE L ANALYSE EXPLORATOIRE - ID IMMOBILIER')
print('=' * 60)
print()
print('VOLUME DE DONNEES')
print(f'  Total annonces     : 5 424 (hors valeurs venales)')
print(f'  Valeurs venales    : 354')
print(f'  TOTAL              : 5 778 enregistrements')
print()
print('QUALITE ESTIMEE')
print(f'  ImmoAsk  : ~80% valides (20% rejets prix m2)')
print(f'  Facebook : ~89% valides (11% rejets prix m2)')
print(f'  CoinAfrique : ~72% valides (28% rejets zones + prix)')
print()
print('OBSERVATIONS CLES')
print('  1. CoinAfrique domine en volume mais a plus de donnees sales')
print('  2. Les prix varient enormement selon le type de bien et la zone')
print('  3. Ecart significatif entre prix marche et valeurs venales OTR')
print('  4. Peu de zones communes entre les 3 sources (fragmentation)')
print('  5. Distribution asymetrique des prix - normale pour l immobilier')
print()
print('PROCHAINE ETAPE : Pipeline de nettoyage (cleaning_v2.py)')
print('=' * 60)